In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


## Roberta Base Model

In [2]:
import os
import shap
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

In [3]:
# setting the model name
PRE_TRAINED_MODEL_NAME = 'roberta-base'

# setting the checkpoint path 
if os.path.exists('ckpts'):
    CHECKPOINTPATH = 'ckpts/Persnality_MBTI'
else:
    CHECKPOINTPATH = '../ckpts/Persnality_MBTI'

# training parameters
MAX_LEN = 512


# TOKENIZER
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

# setting the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# setting the random seed
torch.manual_seed(99)
torch.cuda.manual_seed(99)
torch.cuda.manual_seed_all(99)
np.random.seed(99)

In [4]:
CHECKPOINTPATH

'../ckpts/Persnality_MBTI'

## Model and explainer

In [5]:
class ROBERTAClass(torch.nn.Module):
    def __init__(self, PRE_TRAINED_MODEL_NAME, num_classes=4, dropout=0.3):
        super(ROBERTAClass, self).__init__()
        self.bert_model = AutoModel.from_pretrained(
            PRE_TRAINED_MODEL_NAME, return_dict=True, output_attentions=True)
        self.dropout = torch.nn.Dropout(dropout)
        self.linear = torch.nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

    def getProbabilities(self, input):
        input = input.tolist()
        input = tokenizer(input, max_length=512, truncation=True,
                          padding='longest', return_token_type_ids=True, return_tensors='pt')
        input = {key: tensor.to(device) for key, tensor in input.items()}
        output = self.forward(**input)
        probabilities = torch.sigmoid(output)
        return probabilities

    def getBinaryProbs(self, input):
        probabilities = self.getProbabilities(input)
        IE = []
        NS = []
        TF = []
        JP = []
        # Iterate over each item in the probabilities tensor
        for prob in probabilities:
            # Apply the transformation to each item and directly append to the list
            IE.append(torch.tensor([1 - prob[0], prob[0]]))
            NS.append(torch.tensor([1 - prob[1], prob[1]]))
            TF.append(torch.tensor([1 - prob[2], prob[2]]))
            JP.append(torch.tensor([1 - prob[3], prob[3]]))

        # Convert the list of tensors to a single tensor
        IE = torch.stack(IE)
        NS = torch.stack(NS)
        TF = torch.stack(TF)
        JP = torch.stack(JP)
        return IE, NS, TF, JP
    # ----------------- MODEL PREDICTION -----------------
    def predict(self, text):
        return self.getProbabilities(np.array([text])).round()[0].numpy().tolist()

    # ----------------- SHAP EXPLANATION -----------------
    def explain(self, input, aspect):
        def get_aspect_explanation(input):
            IE, NS, TF, JP = self.getBinaryProbs(input)

            if aspect == 'IE':
                return IE
            elif aspect == 'NS':
                return NS
            elif aspect == 'TF':
                return TF
            elif aspect == 'JP':
                return JP
            else:
                raise ValueError("Invalid aspect provided.")

        class_names = self.get_class_names(aspect)
        aspect_explainer = shap.Explainer(get_aspect_explanation, masker=shap.maskers.Text(), output_names=class_names)
        shap_values = aspect_explainer(input)
        return shap_values
    # ----------------- HELPER FUNCTIONS -----------------
    def get_class_names(self,aspect):
        if aspect == 'IE':
            return ["Introvert", "Extrovert"]
        elif aspect == 'NS':
            return ["Intuition", "Sensing"]
        elif aspect == 'TF':
            return ["Thinking", "Feeling"]
        elif aspect == 'JP':
            return ["Judging", "Perceiving"]
        else:
            raise ValueError("Invalid aspect provided.")

#### Load the model

In [6]:
model = ROBERTAClass(PRE_TRAINED_MODEL_NAME)
model.load_state_dict(torch.load(CHECKPOINTPATH + f'_clean_Best_{PRE_TRAINED_MODEL_NAME}.bin', map_location=torch.device(device)))
model.to(device)
print(f'{PRE_TRAINED_MODEL_NAME}_no_words loaded')
model.eval()

torch.set_grad_enabled(False)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


roberta-base_no_words loaded


In [7]:
import joblib
joblib.dump(model, 'roberta_mbti_model.pkl')

['roberta_mbti_model.pkl']

### Testing the saved model

In [8]:
text="I like to go out and meet new people. I am very outgoing and I enjoy socializing. I am very talkative and I like to be the center of attention." 
text2="I am a very shy person and I prefer to be alone."

In [9]:
roberta_mbti_model = joblib.load('roberta_mbti_model.pkl')
roberta_mbti_model.predict(text)
shap_values = roberta_mbti_model.explain([text,text2], 'IE')
shap.plots.text(shap_values)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 3it [00:20, 10.27s/it]               


In [10]:
roberta_mbti_model.getBinaryProbs(np.array([text]))

(tensor([[0.6275, 0.3725]]),
 tensor([[0.7604, 0.2396]]),
 tensor([[0.4199, 0.5801]]),
 tensor([[0.5336, 0.4664]]))

In [11]:
text="I think people are generally good and I trust them. I am very empathetic and I like to help others. I am very compassionate and I like to make others feel good."
shap_values = roberta_mbti_model.explain([text], 'TF')

  0%|          | 0/498 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:12, 12.41s/it]               


In [12]:
shap.plots.text(shap_values)